# DataKind DC - Red Cross Phase 2
## Smoke Alarm Model 1b: Code reproduction

Maria Brun <br>
April, 2018

In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn import linear_model

#This is old and causes warnings - updated with sklearn.model_selection
# from sklearn.cross_validation import train_test_split, cross_val_score

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import normalize, Imputer
from sklearn.metrics import mean_squared_error
from math import sqrt

#Get the working directory

import os
os.getcwd()

#Change the working directory and confirm it changed to where you want it to be!

os.chdir('C:\\Users\\mcboe72\\Documents\\GitHub\\red-cross-2')
os.getcwd()

'C:\\Users\\mcboe72\\Documents\\GitHub\\red-cross-2'

In [2]:
tract_data = pd.read_csv('https://raw.githubusercontent.com/maria-brun/red-cross-2/master/2009_2013_alarm_presence_by_tract.csv')
del tract_data['Unnamed: 0']
tract_data.head()

,tractid,all_fire_all_years,alarm_unknown_all_years,alarm_presented_all_years,alarm_not_presented_all_years,ratio_no_alarm_in_all,ratio_no_alarm_in_all_known
0,1001020100,4,0,4,0,0.000000,0.000000
1,1001020200,8,1,3,4,0.500000,0.571429
2,1001020300,7,2,0,5,0.714286,1.000000
3,1001020400,8,4,2,2,0.250000,0.500000
4,1001020500,15,5,7,3,0.200000,0.300000


In [4]:
acs = pd.read_csv('http://enigma-public.s3.amazonaws.com/projects/smoke-alarm-risk/data/acs.csv')

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [3]:
acs.head()

NameError: name 'acs' is not defined

The geoid is a code that indicates the summary level, the state, county, etc <br>
For reference, see the documentation at this website: https://www.census.gov/geo/reference/geoidentifiers.html <br> <br>
The number of digits determins the level: 2 for state, 5 for state and county, 10 for state and county and county subdivision, etc <br> <br>
This code splits the column geoid into tract ids - parsed ID and sum level <br>
As an example, I output the full geoid and the parsed ID and sum level for the 100th observation in a list

In [12]:
acs['geoid_parsed'] = [geo[1] for geo in acs['geoid'].str.split('US')]
acs['sum_level'] = [geo[0] for geo in acs['geoid'].str.split('US')]

examples_geoid = (acs['geoid'][100], acs['geoid_parsed'][100], acs['sum_level'][100])
examples_geoid

('15000US060374607001', '060374607001', '15000')

In [13]:
#Creates new variables state, county, etc, by pulling out the part of the geoid that corresponds to each 
#Commented out of code

#acs['state'] = acs['geoid_parsed'].str[0:2]
#acs['cnty'] = acs['geoid_parsed'].str[2:5]
#acs['raw_tract'] = acs['geoid_parsed'].str[5:]

In [14]:
#Create a new data set with just the census tract level summary data

acsCT = acs.query("sum_level == '14000'")
acsCT.head()

,geoid,tenure_renter_occupied,tenure_owner_occupied,vacancy_for_seasonal_recreational,vacancy_for_rent,vacancy_for_sale_only,vacancy_other_vacant,vacancy_rented_not_occupied,vacancy_for_migrant_workers,vacancy_sold_not_occupied,...,hhmove_moved_in_2000_to_2009,hhmove_moved_in_1970_to_1979,hhmove_moved_in_1980_to_1989,hhmove_moved_in_2010_or_later,qfs1_yes,qfs1_no,hdsb_no,hdsb_yes,geoid_parsed,sum_level
32,14000US05015950500,0.356879,0.643121,0.219365,0.124054,0.128593,0.390318,0.027231,0.0,0.110439,...,0.444353,0.057084,0.083368,0.205749,0.198357,0.801643,0.019764,0.034319,05015950500,14000
33,14000US05033020302,0.273465,0.726535,0.259740,0.415584,0.000000,0.324675,0.000000,0.0,0.000000,...,0.421205,0.084589,0.059096,0.201043,0.092700,0.907300,0.014880,0.016411,05033020302,14000
34,14000US05045030702,0.619534,0.380466,0.000000,0.469388,0.178571,0.112245,0.239796,0.0,0.000000,...,0.574344,0.019679,0.032070,0.255102,0.203353,0.796647,0.018452,0.023134,05045030702,14000
35,14000US05057480500,0.503280,0.496720,0.000000,0.222222,0.057143,0.644444,0.076190,0.0,0.000000,...,0.586166,0.038163,0.045319,0.162791,0.191413,0.808587,0.021086,0.032425,05057480500,14000
36,14000US05085020103,0.089406,0.910594,0.212245,0.159184,0.289796,0.220408,0.000000,0.0,0.118367,...,0.459432,0.055297,0.125581,0.121447,0.067183,0.932817,0.008466,0.016740,05085020103,14000


Thsi code imputes the mean for missing values <br>
First, keep only the features (not the target variable) and create a dataframe with just the target variable. Then, use the imputer to replace missing values with the mean. Then merge the target variable back to the transformed features dataframe.

In [15]:
acsCT = acsCT.rename(columns = {'geoid_parsed':'tractid'})

#This code errors 
#acsCT['tractid'] = acsCT['tractid'].astype(int)

#Instead, it is possible to use float, but this seems like an unnecessary step
#acsCT['tractid'] = acsCT['tractid'].astype(float)

#Create target variable only dataframe
acs_tractid = acsCT['tractid']

#Create features variables only dataframe
acsCT = acsCT.drop(['sum_level', 'geoid'], axis = 1)
acs_features = acsCT.drop(['tractid'], axis = 1)
col_names = acs_features.columns

In [17]:
imp = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0).fit(acs_features)

#Renamed dataframe to avoid recursively referencing a dataframe

acs_features_imp = imp.transform(acs_features)

acsCT_imp = pd.DataFrame(acs_features_imp)
acsCT_imp.columns = col_names
acsCT_imp = pd.concat([acsCT_imp.reset_index(drop=True), acs_tractid.reset_index(drop=True)], axis = 1)

#Added to remove duplicate column 'tractID'
#acsCT.drop(acsCT.columns[len(acsCT.columns)-1], axis=1, inplace=True)
acsCT = acsCT_imp.loc[:,~acsCT.columns.duplicated()]

Merge alarm presence by tract data with the census tract level ACS data

In [18]:
tract_data_merged = tract_data.merge(acsCT, how='left', on='tractid')

Filter for all fire years greater than 25

Q!

In [19]:
tract_data_merged.isnull().sum()
tract_data_merged_filtered = tract_data_merged.query(" all_fire_all_years > 25 ")
tract_data_merged_filtered.isnull().sum().head()

tractid                          0
all_fire_all_years               0
alarm_unknown_all_years          0
alarm_presented_all_years        0
alarm_not_presented_all_years    0
dtype: int64

Use resample to bootstrap where all fire years are greater than 50, then append to the tract level data filtered for where all fire years are greater than 25

Q!

In [26]:
from sklearn.utils import resample

high_fires_resampled = resample(tract_data_merged_filtered.query( "all_fire_all_years >= 50" ), n_samples = 1000, random_state = 12)
tract_data_merged_filtered = tract_data_merged_filtered.append(high_fires_resampled)

tract_data_merged_filtered.head()

,tractid,all_fire_all_years,alarm_unknown_all_years,alarm_presented_all_years,alarm_not_presented_all_years,ratio_no_alarm_in_all,ratio_no_alarm_in_all_known,tenure_renter_occupied,tenure_owner_occupied,vacancy_for_seasonal_recreational,...,hhmove_moved_in_1990_to_1999,hhmove_moved_in_1969_or_earlier,hhmove_moved_in_2000_to_2009,hhmove_moved_in_1970_to_1979,hhmove_moved_in_1980_to_1989,hhmove_moved_in_2010_or_later,qfs1_yes,qfs1_no,hdsb_no,hdsb_yes
16,1003010500,32,16,9,7,0.218750,0.437500,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,1003010703,28,16,8,4,0.142857,0.333333,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,1003010705,34,12,11,11,0.323529,0.500000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,1003010800,41,22,12,7,0.170732,0.368421,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,1003011401,38,27,5,6,0.157895,0.545455,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Split the dataframe into features versus target variable dataframes

In [32]:

# These are the explanatory variabels

data_features = tract_data_merged_filtered.drop('ratio_no_alarm_in_all', axis = 1)

data_features = data_features.drop(['tractid', 'alarm_unknown_all_years',
                                    'alarm_presented_all_years', 'alarm_not_presented_all_years',
                                    'ratio_no_alarm_in_all_known'], axis = 1)

#This is the target variable

data_target = tract_data_merged_filtered['ratio_no_alarm_in_all']

#data_features.isnull().sum()

data_features.head()



,all_fire_all_years,tenure_renter_occupied,tenure_owner_occupied,vacancy_for_seasonal_recreational,vacancy_for_rent,vacancy_for_sale_only,vacancy_other_vacant,vacancy_rented_not_occupied,vacancy_for_migrant_workers,vacancy_sold_not_occupied,...,hhmove_moved_in_1990_to_1999,hhmove_moved_in_1969_or_earlier,hhmove_moved_in_2000_to_2009,hhmove_moved_in_1970_to_1979,hhmove_moved_in_1980_to_1989,hhmove_moved_in_2010_or_later,qfs1_yes,qfs1_no,hdsb_no,hdsb_yes
16,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:

# Handle missing observations by simple mean imputation

col_names = data_features.columns

imp2 = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0).fit(data_features)

data_features = imp.transform(data_features)

data_features = pd.DataFrame(data_features)

data_features.columns = col_names

data_features.head()

ValueError: Length mismatch: Expected axis has 1 elements, new values have 250 elements

In [23]:
x_train, x_test, y_train, y_test = train_test_split(data_features,
                                                    data_target,
                                                    test_size=0.1,
                                                    random_state=12)

x_train

,0
105,27.0
291,26.0
12251,58.0
12533,54.0
6216,41.0
1649,29.0
3023,26.0
12218,61.0
10149,32.0
6288,37.0


In [ ]:
all_fires = x_train.all_fire_all_years

observation_weights = x_train.all_fire_all_years/x_train.all_fire_all_years.sum()
observation_weights_normalized = normalize(observation_weights, norm = 'l1').ravel()
scaled_weights = x_train.all_fire_all_years/np.max(x_train.all_fire_all_years)

x_train = x_train.drop(['all_fire_all_years'], axis = 1)
x_test = x_test.drop(['all_fire_all_years'], axis = 1)



# Output to csv
x_train.to_csv('x_train.csv', header = True, index = False)
x_test.to_csv('x_test.csv', header = True, index = False)
y_train.to_csv('y_train.csv', header = True, index = False)
y_test.to_csv('y_test.csv', header = True, index = False)
all_fires.to_csv('all_fires.csv', header = True, index = False)